In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import json
import sys
import re
import logging
import os
from pathlib import Path
from functools import partial
from itertools import chain
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots

import bhutils
from bhutils import bhutilspy as bh
# from api.ezutils import bh, mkt_defs, ccy_cals, mx_cals, mtg_dts, gens

from gioutils.utils import parse_offset, today, get_next_n, get_bday
from api.data.base import data_path, radar_path
from gioutils.gui.base import View, WidgetView, Tabs
from api.gui.params import bbg_params, spot_params, fwd_params
from gioutils.blpw import BlpQuery
from gioutils.utils import RepeatedTimer

from gioutils.ezutils import bh, bhs, SQL, ZCILSHelper, get_price_info, get_id
from api.gui.theme import bg_color, grid_style
from api.gui.pandas import DFOutput, negative_red, custom_styler_row_line, DataGridGio

pd.options.plotting.backend = "plotly"

In [8]:
from api.collectors import SpotForward, Futures, Market, Mtgs, FwdMatrix, VolMatrix
from api.gui.fly_grids import FlyGrid
from api.gui.editable_grid import EditableGrid
from api.gui.price_grids import PricingGrid
from api.gui.tables import SpotTable, FutureTable, GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.grids import SpotGrid, FutureGrid, MtgGrid, MatGrid, GroupMktGrid # GroupMktTalbe, KeyChart, MtgTable, MatTable, CustomGovtBondFutTable
from api.gui.global_config import GlobalConfig
from api.gui.monitors import MktMonitor, VolMonitor

In [9]:
bq = BlpQuery(timeout=50000).start()

In [28]:
from threading import Thread

In [ ]:
bbg_params

In [34]:
def dl(bq, id_=None, res=None):
    print(id_)
    r = bq.bdp('ES1 Index', 'px_last')
    res[id_] = r

In [35]:
threads = []
res = {}
for i in range(2):
    thread = Thread(target=dl, args=(bq, i, res), daemon=True)
    thread.start()
    threads.append(thread)

0
1


In [33]:
res

{1:     security  px_last
 0  ES1 Index  3960.25,
 0:     security  px_last
 0  ES1 Index  3960.25}

In [20]:
thread = Thread(target=dl, args=(3, ))

In [16]:
threads[1].name

'Thread-6 (dl)'

In [5]:
ccys = [
    'EUR', 
    'USD', 
    'GBP'
]

In [6]:
curves_df = pd.DataFrame(curves).set_index('ccy').loc[ccys]

In [8]:
pricer_grid = PricingGrid()
mm = MktMonitor()
vm = VolMonitor()

In [9]:
dc = DataCollector(
    curves=curves_df.melt()['value'].drop_duplicates().dropna().to_list(),
    bq=bq,
    collectors={
        # 'fg': fly_grid,
        'pg': pricer_grid,
        'sf': SpotForward(spot_params=spot_params, fwd_params=fwd_params),
        'fut': Futures(),
        'mkt': Market(),
        'mtgs': Mtgs(),
        'mat': FwdMatrix(),
        'vol_mat': VolMatrix(),
    },
    renderers={
        'mm': mm,
        'vm': vm
    }
)

In [10]:
gc = GlobalConfig(dc=dc)

In [15]:
tabs = Tabs(
    {
        'radar': v.Container(
            fluid=True,
            children=[
                v.Row(children=[v.Col(children=[gc.view])]),
                v.Row(children=[v.Col(children=[mm.view])])
            ],
        ),
        'vol':v.Container(
            fluid=True,
            children=[vm.view]
        ),
        'pricer': v.Container(
            fluid=True,
            children=[w.VBox([pricer_grid.box, pricer_grid.price_grid.view])],
        ),
        'monitor': pricer_grid.expr_dg.view,
    },
    force_grid_render=True,
    # tabs_kwargs={'class_': "mb-2"}
)
tabs

Card(children=[Tabs(children=[Tab(children=['radar']), Tab(children=['vol']), Tab(children=['pricer']), Tab(ch…

In [ ]:
self = eco

In [ ]:
tickers = self._params['security']
fields = self._fields

In [ ]:
release_date_field = 'ECO_FUTURE_RELEASE_DATE_LIST'

In [ ]:
ref_data = bq.bdp(
    tickers,
    fields
)

In [ ]:
hist_data = bq.bdh(
    tickers,
    {'actual_release': 'actual', 'first_revision': 'rev'},
    today() - parse_offset('ytd'),
    options=False
)

In [ ]:
relase_dates = bq.bdsm(
    securities=tickers,
    field=release_date_field
).droplevel(1).droplevel(1, axis=1).rename({release_date_field: 'date'}, axis=1).assign(release=True)

In [ ]:
new_data = None

[autoreload of api.gui.grids failed: Traceback (most recent call last):
  File "C:\dev\projects\menashe\venv\lib\site-packages\IPython\extensions\autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "C:\dev\projects\menashe\venv\lib\site-packages\IPython\extensions\autoreload.py", line 459, in superreload
    module = reload(module)
  File "C:\Users\gperciaccante\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "C:\dev\projects\menashe\api\gui\grids.py", line 550
    def _make_default_
                      ^
SyntaxErr

In [ ]:
new_tickers = ref_data.query(f"next == '{today():%Y-%m-%d}'")['security']
new_fields = {
    'eco_release_dt': 'next',
    'actual_release': 'actual',
    'bn_survey_median': 'survey',
}
new_data = bq.bdp(
    new_tickers,
    new_fields
)

data = {
    'ref': ref_data,
    'hist': hist_data,
    'dates': relase_dates,
    'new': new_data
}

9      GDP CQOQ Index
13       ITCPEY Index
15      CPMINDX Index
19        AUBAC Index
27     MBAVCHNG Index
38     ADP CHNG Index
50     GRUECHNG Index
52     ECCPEMUM Index
57     CHPMINDX Index
60     MWINCHNG Index
66     GDP PIQQ Index
69     CPMINMAN Index
72     USPHTMOM Index
77      ECCPEST Index
90     GDPCPCEC Index
91     GDPCTOT% Index
97       GRUEPR Index
98     CPEXEMUY Index
119    FRCPEECY Index
Name: security, dtype: object

In [32]:
var = 'NFP TCH Index'

In [33]:
one_var = hist_data.query(f"security == '{var}'")

In [34]:
d1, d2 = today() - parse_offset('')

SyntaxError: invalid syntax (4129968472.py, line 1)

In [27]:
relase_dates.loc[var]

field                         date
security                          
NFP TCH Index  2021/01/08 13:30:00
NFP TCH Index  2021/02/05 13:30:00
NFP TCH Index  2021/03/05 13:30:00
NFP TCH Index  2021/04/02 13:30:00
NFP TCH Index  2021/05/07 13:30:00
NFP TCH Index  2021/06/04 13:30:00
NFP TCH Index  2021/07/02 13:30:00
NFP TCH Index  2021/08/06 13:30:00
NFP TCH Index  2021/09/03 13:30:00
NFP TCH Index  2021/10/08 13:30:00
NFP TCH Index  2021/11/05 12:30:00
NFP TCH Index  2021/12/03 13:30:00
NFP TCH Index  2022/01/07 13:30:00
NFP TCH Index  2022/02/04 13:30:00
NFP TCH Index  2022/03/04 13:30:00
NFP TCH Index  2022/04/01 13:30:00
NFP TCH Index  2022/05/06 13:30:00
NFP TCH Index  2022/06/03 13:30:00
NFP TCH Index  2022/07/08 13:30:00
NFP TCH Index  2022/08/05 13:30:00
NFP TCH Index  2022/09/02 13:30:00
NFP TCH Index  2022/10/07 13:30:00
NFP TCH Index  2022/11/04 12:30:00
NFP TCH Index  2022/12/02 13:30:00
NFP TCH Index  2023/01/06 13:30:00
NFP TCH Index  2023/02/03 13:30:00
NFP TCH Index  2023/03/10 13:30:00
NFP TCH Index  2023/04/07 13:30:00
NFP TCH Index  2023/05/05 13:30:00
NFP TCH Index  2023/06/02 13:30:00
NFP TCH Index  2023/07/07 13:30:00
NFP TCH Index  2023/08/04 13:30:00
NFP TCH Index  2023/09/01 13:30:00
NFP TCH Index  2023/10/06 13:30:00
NFP TCH Index  2023/11/03 12:30:00
NFP TCH Index  2023/12/08 13:30:00